In [0]:
!pip install tensornetwork

     |████████████████████████████████| 122kB 4.8MB/s 
     |████████████████████████████████| 71kB 16.2MB/s 
  Created wheel for opt-einsum: filename=opt_einsum-3.0.0-cp36-none-any.whl size=58490 sha256=8669c40a8faaff143080659a475574769dd84b63268f604b06c969975ff113a9
  Stored in directory: /root/.cache/pip/wheels/a1/82/aa/e6d68e36a66706a69f890a6072793d2b87730572499c626ed8
Successfully built opt-einsum
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
import numpy as np
from numpy import linalg as LA
import tensorflow as tf
import tensornetwork
from tensornetwork import ncon
net = tensornetwork.TensorNetwork()
sess = tf.InteractiveSession()

# Descomposicion tensorial multi-etapas o SVD truncado

¿Como podemos descomponer adecuadamente un tensor $H$ de muchos indices en una red $T$ de tensores $\{A,B,C,...\}$ de acuerdo a una geometria dada? Minimizando $||H-T||$ dada una dimension $\chi$ de los indices internos de $T$

<img src="https://static.wixstatic.com/media/d91e93_1d7d73f53f2e4183aa2165660f6f7358~mv2.png/v1/fill/w_638,h_225,al_c,q_90/fig1.webp" width="400">

Debemos emplear <u>descomposicion tensorial multi-etapas</u> que es una secuencia de descomposiciones SVD. La manera correcta es que el tensor a ser descompuesto en cada etapa debe ser un centro de ortogonalidad, para asegurar que el error global de truncamiento se minimice.

**Ejemplo**

<img src="https://static.wixstatic.com/media/d91e93_01e52a5f9e3f485298f08437d570bc3c~mv2.png/v1/fill/w_950,h_591,al_c,q_90/fig2.webp" width="500">

In [0]:
##### Descomposicion multi-etapas
d = 5 # dimension local
chi = 3 # maxima dimension interna
H0 = np.random.rand(d,d,d,d,d,d,d) 

# primera descomposicion
utemp,stemp,vhtemp = LA.svd(H0.reshape(d**2,d**5),full_matrices=False)
U0 = (utemp[:,:chi]).reshape(d,d,chi)
H1 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,d,d,d,d,d)
# segunda descomposicion
utemp,stemp,vhtemp = LA.svd(H1.transpose(1,2,0,3,4,5).reshape(d**2,chi*d**3),full_matrices=False)
U1 = (utemp[:,:chi]).reshape(d,d,chi)
H2 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,chi,d,d,d).transpose(1,0,2,3,4)
# tercera descomposicion
utemp,stemp,vhtemp = LA.svd(H2.reshape(chi**2,d**3),full_matrices=False)
U2 = (utemp[:,:chi]).reshape(chi,chi,chi)
H3 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,d,d,d)
# cuarta descomposicion
utemp,stemp,vhtemp = LA.svd(H3.reshape(chi*d,d**2),full_matrices=False)
V3 = vhtemp[:chi,:].reshape(chi,d,d).transpose(1,2,0)
H4 = (utemp[:,:chi] @ np.diag(stemp[:chi])).reshape(chi,d,chi)
# verificacion
H0recovered = ncon([U0,U1,U2,V3,H4],[[-1,-2,1],[-3,-4,2],[1,2,3],[-6,-7,4],[3,-5,4]])
print("totErr = ",LA.norm(H0 - H0recovered.eval()) / LA.norm(H0))

totErr =  0.4978139421508944


# Centro de ortogonalidad (centrado en enlace)

Fijamos un indice interno (o enlace) dentro de la red como un centro de ortogonalidad.


1.   Introducimos una matriz de enlace $\sigma$ situada en el enlace en cuestion que va a ser definida inicialmente como la matriz identidad (no cambia las propiedades de la red).
2.   Luego, usamos algunos de los metodos vistos para fijar $\sigma$ como centro de ortogonalidad.



<br>
<b>Ejemplo:</b> Uso del metodo de ortogonalizacion directa en el enlace $B$-$C$ para establecerlo como centro de ortogonalidad de la red $\{A,B,C\}$

<img src="https://static.wixstatic.com/media/d91e93_1d0acbb1faea468daf2a5da7e96ab211~mv2.png/v1/fill/w_900,h_554,al_c,q_90/fig3.webp" width="500">

In [0]:
##### Establecer enlace B-C como centro de ortogonalidad
d = 5 # dimension de indices
A = np.random.rand(d,d,d)
B = np.random.rand(d,d,d)
C = np.random.rand(d,d,d)
Sig = np.eye(d); # matriz inicial (identidad) del enlace

# generar un cambio gauge de las matrices
rho1 = ncon([A,A,B,B],[[1,2,3],[1,2,4],[3,5,-1],[4,5,-2]])
rho2 = ncon([C,C],[[-1,1,2],[-2,1,2]])
d1, u1 = LA.eigh(rho1.eval()); sq_d1 = np.sqrt(abs(d1))
d2, u2 = LA.eigh(rho2.eval()); sq_d2 = np.sqrt(abs(d2))
X1 = u1 @ np.diag(sq_d1) @ u1.T; X1inv = u1 @ np.diag(1/sq_d1) @ u1.T
X2 = u2 @ np.diag(sq_d2) @ u2.T; X2inv = u2 @ np.diag(1/sq_d2) @ u2.T
# implementar cambio gauge
Bprime = ncon([B,X1inv],[[-1,-2,1],[1,-3]])
Cprime = ncon([X2inv,C],[[-1,1],[1,-2,-3]])
Sig_prime = X1 @ Sig @ X2
# verificar
H0 = ncon([A,B,C],[[-1,-2,1],[1,-3,2],[2,-4,-5]])
H1 = ncon([A,Bprime,Sig_prime,Cprime],[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]])
print("totErr = ",LA.norm(H0.eval() - H1.eval()) / LA.norm(H0.eval()))

totErr =  1.2900867616322735e-15


Hay que imponer una restriccion adicional: el gauge debe ser elegido de forma que al final la matriz de enlace sea diagonal con elementos positivos en orden descendiente de magnitud. 
Se necesita un paso final en donde se hace la descomposicion SVD de $\sigma'$ y luego contraemos las matrices $U$ y $V^†$ con los tensores que conecta $\sigma'$ para dejarla como matriz diagonal.

<img src="https://static.wixstatic.com/media/d91e93_301718cb104b441c9e6528490cbc2c7e~mv2.png/v1/fill/w_825,h_413,al_c,q_90/fig4.webp" width="500">

In [0]:
##### Establecer un enlace como centro de ortogonalidad
##### diagonalizacion de matriz de enlace

# Realizar cambio gauge unitario para diagonalizar la matriz de enlace
utemp, Sig_pp, vhtemp = LA.svd(Sig_prime,full_matrices=False)
Bpp = ncon([Bprime,utemp],[[-1,-2,1],[1,-3]])
Cpp = ncon([Cprime,vhtemp],[[1,-2,-3],[-1,1]])
# verificar
H2 = ncon([A,Bpp,np.diag(Sig_pp),Cpp],[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]])
print("totErr = ",LA.norm(H0.eval() - H2.eval()) / LA.norm(H0.eval()))

totErr =  9.104059304245175e-16


## Equivalencia del (enlace) centro de ortogonalidad y el SVD

El resultado de fijar un enlace dentro de una red como centro de ortogonalidad es equivalente a un SVD: la matriz de enlace resultante $\sigma''$ contiene los valores singulares, mientras que las redes a la izquierda / derecha del enlace son equivalentes bajo contracción a las isometrías $U / V$.

<img src="https://static.wixstatic.com/media/d91e93_994b998eb81448248c6a343c0b2b874e~mv2.png/v1/fill/w_888,h_435,al_c,q_90/fig6.webp" width="500">




# Formas Canonicas o Normal

Al establecer un enlace como centro de ortogonalidad, la restriccion de que la matriz del enlace sea diagonal puede fijar completamente la invariancia gauge de este enlace (En general, esto es verdadero si los elementos de la matriz de enlace son no degenerados).  Esta propiedad de unicidad se deriva de la correspondencia entre el centro de ortogonalidad basado en enlaces y el SVD, que es igualmente único.
<br>Una red está en forma canónica, si todas las matrices de enlaces son simultáneamente centros de ortogonalidad, fijando de forma unica el gauge en cada uno de los enlaces de la red.

### Procedimiento para transformar una red a forma canónica:

*    Coloque una matriz de enlace $\sigma$, inicialmente establecida como matriz de identidad, en cada índice interno.

*    Use el enfoque de ortogonalización directa para establecer independientemente cada matriz de enlace $\sigma$ como un centro de ortogonalidad.

*    Realice una descomposicion SVD a cada enlace para que sea una matriz diagonal con elementos positivos ordenados descendientemente.

<img src="https://static.wixstatic.com/media/d91e93_89ebeadd221a42e0860ca130d32e422f~mv2.png/v1/fill/w_956,h_563,al_c,q_90/fig7.webp" width="500">


<br>

<b>Ejemplo:</b> La red se supone que está en forma canónica. 
<br><img src="https://static.wixstatic.com/media/d91e93_5aa23c4a812449578d674eb841eb6c0f~mv2.png/v1/fill/w_750,h_445,al_c,q_90/fig8.webp" width="400">

1.    Suponiendo que deseamos establecer algún tensor específico $E$ como el centro de ortogonalidad, primero etiquetamos cada índice con una flecha que apunta hacia el tensor $E$. 
2.    Luego absorbemos en cada tensor cualquier matriz de enlace $\sigma$ que reside en su entrada de índices. 
3.    De las propiedades de la forma canónica se deduce que todos los tensores en la red resultante ahora son isométricos con respecto a la partición entre sus índices de tensor entrantes y salientes, similar a la red producida por el enfoque de 'Pulling Through'. Además, el tensor $E$ especificado es un centro de ortogonalidad.

###Resumen: Forma canonica

La forma canónica resulta de una forma particular de fijar los grados de libertad del gauge en una red tensorial, y es útil ya que organiza una red de tal manera que la información importante esté fácilmente disponible. Al configurar una red en forma canónica, uno puede:

*    truncar de manera óptima cualquier enlace a una dimensión reducida, simplemente descartando los valores singulares más pequeños de la matriz de enlaces correspondiente, y comprender fácilmente el error de truncamiento correspondiente.
*    esencialmente elimina la ambigüedad del gauge de la descripción de la red, debido a la singularidad de la forma canónica.
*    extraer ciertos tipos de información de la red (como los valores esperados) de manera simplificada, debido a las cancelaciones que surgen en redes que contienen tensores isométricos.